In [8]:
import pandas as pd

# Ruta del archivo .txt y nombre del archivo .csv de salida
nombres_txt = {"basal":["R1","R2","R3"],"ejercicio":["derivada1","derivada2","derivada3"],"respiracion":["derivada1","derivada2","derivada3"]}
nombres_csv = {"basal":["Basal_DI","Basal_DII","Basal_DIII"],
                "ejercicio":["Ejercicio_DI","Ejercicio_DII","Ejercicio_DIII"],
                "respiracion":["Respiracion_DI","Respiracion_DII","Respiracion_DIII"]}

ruta_a_txts = "Señal_ecg/"
ruta_a_csvs = "archivos_csv/"
for gesto in nombres_txt.keys():
    contador = 0
    for derivada in nombres_txt[gesto]:
        ruta_completa = ruta_a_txts+gesto+"/"+derivada+".txt"
        with open(ruta_completa,"r") as archivo:
            data = obtener_dato_de_txt(archivo)
        data = filtro_rechaza_banda(data,1000,59,61,4)
        tabla = pd.DataFrame(data)
        tabla.to_csv(ruta_a_csvs+nombres_csv[gesto][contador]+".csv",header=False, index=False)
        contador += 1



## Función para leer data

In [7]:
#Función para obtener datos del TXT
#solo la data de interés (la 5ta columna)
def obtener_dato_de_txt(archivo):
    datos = []
    marcador = False
    #empezar la lactura de los txt
    for linea in archivo: #por cada línea
        if marcador: #recién adjuntará datos al ser verdadero
            datos.append(linea.rstrip().split("\t")[5])
        elif linea.rstrip() == "# EndOfHeader":  #marcador será verdadero a partir la línea esa
            marcador = True #por lo tanto, solo nos devolverá los datos, sin los comentarios
    return datos

## Para filtro notch

In [6]:
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt

# Parámetros: 
    #orden 4
    #butterworth
    #fc1 = 59Hz
    #fc2 = 61Hz
def filtro_rechaza_banda(signal, fs, fc1, fc2, order):
    # Calcular frecuencias normalizadas
    nyquist = fs / 2  # Frecuencia de Nyquist
    low = fc1 / nyquist  # Frecuencia normalizada baja
    high = fc2 / nyquist  # Frecuencia normalizada alta

    # Diseño del filtro Butterworth Rechaza Banda
    b, a = butter(order, [low, high], btype='bandstop')

    # Aplicar el filtro
    #signal es una lista de str
    new_signal = []
    for dato in signal:
        new_signal.append(float(dato))
    filtered_signal = filtfilt(b, a, new_signal)

    return filtered_signal